In [14]:
import numpy as np
import time
import os

AttributeError: module 'numpy' has no attribute 'ndarray'

In [2]:
current_path = os.getcwd()
filename = os.path.join(current_path, "matrix_100x100.csv")
matrix1 = np.genfromtxt(filename, delimiter=',').astype(np.float32)
print(np.shape(matrix1),np.array_equal(matrix1, matrix1.T))

filename = os.path.join(current_path, "matrix_200x200.csv")
matrix2 = np.genfromtxt(filename, delimiter=',').astype(np.float32)
print(np.shape(matrix2),np.array_equal(matrix2, matrix2.T))

filename = os.path.join(current_path, "matrix_400x400.csv")
matrix4 = np.genfromtxt(filename, delimiter=',').astype(np.float32)
print(np.shape(matrix4),np.array_equal(matrix4, matrix4.T))

filename = os.path.join(current_path, "matrix_800x800.csv")
matrix8 = np.genfromtxt(filename, delimiter=',',).astype(np.float32)
print(np.shape(matrix8),np.array_equal(matrix8, matrix8.T))

(100, 100) True
(200, 200) True
(400, 400) True
(800, 800) True


In [3]:
def norm(v):
    return np.sqrt(np.dot(v,v))

def normalize(v):
    return v/norm(v)

In [4]:
def qr_householder(A):
    m, n = A.shape
    R = A.astype(float)  # Avoid unnecessary copy, ensure mutability
    Q = np.eye(m)

    for i in range(n - 1):
        # Compute the Householder vector
        u = R[i:, i].copy()  # Work with a slice; make a copy to modify
        alpha = norm(u)
        u[0] -= alpha  # Create the Householder vector in place
        norm_u = norm(u)

        if norm_u != 0:  # Avoid division by zero
            u /= norm_u

            # Apply the reflection to R in place
            R[i:, i:] -= 2 * np.outer(u, u @ R[i:, i:])
            
            # Apply the reflection to Q in place
            Q[:, i:] -= 2 * np.outer(Q[:, i:] @ u, u)
    return Q, R


In [5]:
def createRQ(A):
    Q, R = qr_householder(A)
    B = np.matmul(R, Q)
    return B, Q

In [6]:
epsilon = 1e-8  # Using exponentiation
MAX_ITERATIONS = 1000

In [7]:
def qr_algorithm_with_shifts(A, refeig=-1, sindex=-1):
    # Compute the initial QR decomposition of A
    B, Q = createRQ(A)
    
    n = 0  # Iteration counter
    leig = B[refeig, refeig]  # Initial reference eigenvalue
    diff = 1  # Difference tracker for convergence

    # Iterate until the eigenvalues stabilize (very small difference)
    while diff > epsilon and n < MAX_ITERATIONS:
        shift = np.eye(len(B)) * B[sindex, sindex]  # Create shift matrix
        C = B - shift  # Apply shift to B
        B, Q = createRQ(C)  # Perform QR decomposition
        
        for i in range(len(B)): # Undo the shift
            B[i, i] += shift[i, i]  

        n += 1  # Count iterations
        diff = abs(leig - B[refeig, refeig])  # Compute change in eigenvalue
        leig = B[refeig, refeig]  # Update reference eigenvalue

    # Extract eigenvalues (diagonal elements of B)
    eigs = [B[i, i] for i in range(len(B))]
    
    # Extract eigenvectors (columns of Q)
    eigv = [Q[:, i] for i in range(Q.shape[1])]
    return eigs, eigv



In [ ]:
def qr_algorithm_with_shifts(A, epsilon=1e-6, max_iterations=1000):
    """Computes the eigenvalues and eigenvectors of A using the QR algorithm with shifts."""
    n = A.shape[0]
    B = A.copy()
    Q_total = np.eye(n)  # Store accumulated transformations

    # Initial eigenvalue estimate (bottom-right)
    leig = B[-1, -1]
    diff = 1
    iterations = 0

    while diff > epsilon and iterations < max_iterations:
        # Apply Wilkinson shift (last diagonal entry)
        shift = np.eye(n) * B[-1, -1]
        C = B - shift  # Apply shift
        B, Q = createRQ(C)  # Compute QR decomposition
        B = B + shift  # Undo shift

        # Accumulate Q transformations for eigenvectors
        Q_total = Q_total @ Q

        iterations += 1
        diff = abs(leig - B[-1, -1])  # Change in eigenvalue
        leig = B[-1, -1]  # Update eigenvalue estimate

        # Check if matrix is upper triangular (converged)
        off_diag_norm = np.sqrt(np.sum(np.triu(B, k=1)**2))
        if off_diag_norm < epsilon:
            break

    # Extract eigenvalues (diagonal of B)
    eigenvalues = np.diag(B)

    # Extract eigenvectors (columns of accumulated Q)
    eigenvectors = Q_total

    return eigenvalues, eigenvectors

In [13]:
matrix = [[6, 2, 1], [2, 5, 3], [1, 3, 4]]
number_of_eigenvectors_for_print = 5

In [14]:
# Measure time for qr_shifts
start_time = time.time()
eigs_qr, eigv_qr = qr_algorithm_with_shifts(matrix, refeig=-1, sindex=-1)
qr_time = time.time() - start_time
print(f"\nExecution time for qr_shifts: {qr_time:.6f} seconds")

# Measure time for NumPy's eig function
start_time = time.time()
eigs_np, eigv_np = np.linalg.eig(matrix)
np_time = time.time() - start_time
print(f"Execution time for np.linalg.eig: {np_time:.6f} seconds")


Execution time for qr_shifts: 11.760059 seconds
Execution time for np.linalg.eig: 0.638535 seconds


In [15]:
# Sort and compare the results
eigs_qr_sorted = sorted(eigs_qr)
eigs_np_sorted = sorted(eigs_np)

print("\nEigenvalues from qr_shifts:")
print(eigs_qr_sorted)

print("\nEigenvalues from np.linalg.eig:")
print(eigs_np_sorted)

# Check the maximum difference between computed eigenvalues
max_diff = max(abs(np.array(eigs_qr_sorted) - np.array(eigs_np_sorted)))
print(f"\nMax difference between methods: {max_diff}")





Eigenvalues from qr_shifts:
[np.float64(-2.6228162089762765), np.float64(-2.248675144918251), np.float64(-1.549549973052186), np.float64(-1.537825372209126), np.float64(-1.4596847111615772), np.float64(-1.3956963932223756), np.float64(-1.3909246507693234), np.float64(-1.3863557386989835), np.float64(-1.3742133150495244), np.float64(-1.364513962392597), np.float64(-1.337160316088027), np.float64(-1.3269187290988904), np.float64(-1.3110305199487378), np.float64(-1.2521329835595678), np.float64(-1.2277630460710116), np.float64(-1.20925103585469), np.float64(-1.206949390876406), np.float64(-1.203992693600343), np.float64(-1.1561427667390474), np.float64(-1.1525067388181658), np.float64(-1.1422857610675026), np.float64(-1.1313804816187827), np.float64(-1.0805934740759584), np.float64(-1.054856610509153), np.float64(-1.0515660685937007), np.float64(-1.0372886236291865), np.float64(-1.0233390290406115), np.float64(-1.0069518759464673), np.float64(-0.9841590970369168), np.float64(-0.983321337

In [11]:
# Print eigenvectors
print(f"\nFirst {number_of_eigenvectors_for_print} eigenvectors from qr_shifts:")
for i in range(number_of_eigenvectors_for_print):
    print(eigv_qr[i])

print(f"\nFirst {number_of_eigenvectors_for_print} eigenvectors from np.linalg.eig:")
for i in range(number_of_eigenvectors_for_print):
    print(eigv_np[:, i])


First 5 eigenvectors from qr_shifts:
[ 9.99998601e-01  9.22602538e-04  9.20065381e-04  6.06361395e-04
  4.84117765e-04  3.09790846e-04  3.35873417e-04  7.36289126e-05
  2.26935953e-04  2.63819251e-04  1.37928675e-04  1.40602734e-04
  1.67085769e-04 -9.52579094e-06  2.56528524e-05  1.68476166e-04
  8.65534154e-05 -5.24244440e-05  4.28882578e-05  7.31656013e-05
  6.05898421e-05  7.44745482e-05  2.21848594e-05  6.72004887e-05
  1.00385207e-04  1.01482993e-04  1.53444678e-05  2.49802787e-05
  1.81953099e-05  3.52185059e-05  7.53289360e-06  1.23333194e-05
 -1.34761099e-05 -5.53706661e-06 -7.20514675e-07  3.66293623e-05
  5.34829023e-05  2.73175127e-05  4.31728413e-05  2.10635596e-05
  1.68941615e-05  1.26200397e-05  3.23311843e-05  3.24354270e-06
 -3.57391090e-06 -6.68982796e-06  2.39304960e-05  1.26645744e-05
  9.61150731e-06  2.32372625e-05 -1.76288246e-05  2.35179837e-05
  2.71122731e-05 -4.25789502e-06  1.03952955e-05 -3.60989918e-06
 -1.42699832e-06  9.78293332e-06 -1.67564365e-06  6.

In [12]:
Q, R = qr_householder(matrix1)
Q

array([[ 0.1704072 , -0.04374956,  0.02605791, ...,  0.07964443,
        -0.06704256, -0.05263242],
       [ 0.11675481, -0.09080641,  0.06914678, ..., -0.02327563,
        -0.06378921,  0.01410436],
       [ 0.14623   , -0.01439259, -0.16655938, ..., -0.04353737,
         0.09048337, -0.25513044],
       ...,
       [ 0.08986564,  0.135988  ,  0.02406309, ...,  0.01922851,
        -0.1152549 , -0.23660022],
       [ 0.1599904 , -0.09838831,  0.0433653 , ..., -0.03035425,
         0.19517043,  0.12119133],
       [ 0.10520587,  0.04849107,  0.01103211, ...,  0.12479572,
         0.10281792, -0.09715307]])